In [21]:
# TODO: filled in dataset path, e.g. '/demo-mount/datasets/cyclist'
cyclist_dataset_path = None
# cyclist_dataset_path = '/demo-mount/datasets/cyclist'

# TODO: filled in output voc dataset path, e.g. '/demo-mount/datasets/cyclist-voc'
output_dataset_path = None
# output_dataset_path = '/demo-mount/datasets/cyclist-voc'

# TODO: filled cs231n project directory, e.g. '/home/lizhe/cs231n_project'
cs231n_project_path = None
# cs231n_project_path = '/home/lizhe/cs231n_project'

# TODO: filled in the output kitti format eval data, e.g. '/demo-mount/datasets/cyclist-kitti'
# eval_kitti library need a kitti format data as input, so we need to convert the validation and test data
# from tsinghua dataset to kitti format.
output_kitti_format_eval_data_path = None
# output_kitti_format_eval_data_path = '/demo-mount/datasets/cyclist-kitti'

assert cyclist_dataset_path is not None
assert output_dataset_path is not None
assert cs231n_project_path is not None
assert output_kitti_format_eval_data_path is not None

training_subdir = 'leftImg8bit/train/tsinghuaDaimlerDataset/'
validation_subdir = 'leftImg8bit/valid/tsinghuaDaimlerDataset/'
test_subdir = 'leftImg8bit/test/tsinghuaDaimlerDataset/'

In [76]:
%%bash -s "$cyclist_dataset_path" "$training_subdir" "$validation_subdir" "$test_subdir"

cd $1
ls -1 $2 | cut -d. -f1 > train.txt
ls -1 $3 | cut -d. -f1 > valid.txt
ls -1 $4 | cut -d. -f1 > test.txt

# head -500 full_train.txt > train.txt
# head -500 full_valid.txt > valid.txt
# head -500 full_test.txt > test.txt

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [77]:
%%bash -s "$cyclist_dataset_path" "$output_dataset_path" "$cs231n_project_path"

# This takes around 30 minutes to finish
cd $3/vod-converter
python vod_converter/main.py --from tsinghua-daimler --from-path $1 --to voc --to-path $2

Successfully converted from tsinghua-daimler to voc.


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
INFO:root:Namespace(filter_images_without_labels=False, from_key='tsinghua-daimler', from_path='/demo-mount/datasets/cyclist', select_only_known_labels=False, to_key='voc', to_path='/demo-mount/datasets/cyclist-voc')


In [78]:
%%bash -s "$cyclist_dataset_path" "$output_dataset_path"
# Generate train.txt, valid.txt and test.txt

cp $1/train.txt $2/VOC2012/ImageSets/Main/raw_train.txt
cp $1/valid.txt $2/VOC2012/ImageSets/Main/raw_valid.txt
cp $1/test.txt $2/VOC2012/ImageSets/Main/raw_test.txt

cd $2/VOC2012/ImageSets/Main/

sed 's/_leftImg8bit//g' raw_train.txt > train.txt
sed 's/_leftImg8bit//g' raw_valid.txt > valid.txt
sed 's/_leftImg8bit//g' raw_test.txt > test.txt

rm raw_train.txt
rm raw_valid.txt
rm raw_test.txt

head -100 train.txt > small_train.txt
head -10 valid.txt > small_valid.txt
head -10 test.txt > small_test.txt

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
chdir: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [79]:
%%bash -s "$cs231n_project_path" "$output_dataset_path"
# Create linkage for detectron2 to load the data
# Supposely the error can be ignored, you can check whether the linkage is correctly created in the printed directory.

mkdir -p $1/detectron2-ResNeSt/datasets/tsinghua_cyclist
ln -s $2/VOC2012 $1/detectron2-ResNeSt/datasets/tsinghua_cyclist/VOC2012
echo $1/detectron2-ResNeSt/datasets/tsinghua_cyclist/VOC2012

/home/lizhe/cs231n_project/detectron2-ResNeSt/datasets/tsinghua_cyclist/VOC2012


shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
ln: failed to create symbolic link '/home/lizhe/cs231n_project/detectron2-ResNeSt/datasets/tsinghua_cyclist/VOC2012/VOC2012': File exists


In [28]:
%%bash -s "$cyclist_dataset_path" "$output_kitti_format_eval_data_path" "$cs231n_project_path" "$training_subdir" "$validation_subdir" "$test_subdir"

cd $1
ls -1 $4 | cut -d. -f1 > full_train.txt
ls -1 $5 | cut -d. -f1 > full_valid.txt
ls -1 $6 | cut -d. -f1 > full_test.txt

# No need to convert training
head -0 full_train.txt > train.txt
# Convert all fo the validation data and test data
cp full_valid.txt valid.txt
cp full_test.txt test.txt

# head -0 full_train.txt > train.txt
# head -10 full_valid.txt > valid.txt
# head -10 full_test.txt > test.txt

cd $3/vod-converter
mkdir -p /demo-mount/datasets/cyclist-kitti
python vod_converter/main.py --from tsinghua-daimler --from-path $1 --to kitti --to-path $2

cd $1
cp full_train.txt train.txt
cp full_valid.txt valid.txt
cp full_test.txt test.txt

start ingesting source data...
ingesting 1000/1019 images in valid subdir
ingesting 1000/2914 images in test subdir
ingesting 2000/2914 images in test subdir
finish ingesting source data!
validated source data! start converting...
converting 1000/3933...
converting 2000/3933...
converting 3000/3933...
Finished converting! Start egesting to output...
Successfully converted from tsinghua-daimler to kitti.


INFO:root:Namespace(filter_images_without_labels=False, from_key='tsinghua-daimler', from_path='/demo-mount/datasets/cyclist', select_only_known_labels=False, to_key='kitti', to_path='/demo-mount/datasets/cyclist-kitti-small')


In [29]:
%%bash -s "$output_kitti_format_eval_data_path" "$cs231n_project_path"
mkdir $2/eval_kitti/build/data/object -p
ln -s $1/training/label_2 $2/eval_kitti/build/data/object/label_2

In [32]:
%%bash -s "$cs231n_project_path"

# Rebuild eval_kitti as there's code change
cd $1/eval_kitti/build
cmake ..
make

-- Configuring done
-- Generating done
-- Build files have been written to: /home/lizhe/cs231n_project/eval_kitti/build
Scanning dependencies of target evaluate_object
[ 50%] Building CXX object CMakeFiles/evaluate_object.dir/evaluate_object.cpp.o
[100%] Linking CXX executable evaluate_object
[100%] Built target evaluate_object
